In [1]:
import os
import json
import numpy as np
import pickle
import pandas as pd
import geopandas
import matplotlib.pyplot as plt

In [2]:
file = "fua.gpkg"
fua = geopandas.read_file(os.path.join("data", "ghsl",file))
fua = fua.to_crs("EPSG:4326")

In [5]:
def get_fua(fua, city):
    '''Returns the corresponding FUA given a city name

    Args:
        fua (geopandas dataframe): GHSL dataset
        city (string): City name with capital first letter, eg. "Paris"
    '''
    result = fua[fua.eFUA_name==city]
    if result is None:
        print("FUA not found")
    return result

In [13]:
cairo = get_fua(fua, 'Cairo')

In [87]:
x = cairo.geometry.to_numpy()[0]
y = x.geoms[0].exterior.coords

AttributeError: 'CoordinateSequence' object has no attribute 'to_json'

In [107]:
def write_json_fua(filename, fua, cities):
    '''Populate a JSON file with the bounding boxes of cities FUA's

    Args:
        filename (string): json filename
        fua (geopandas dataframe): fua dataframe
        cities (list): list of cities names
    '''
    data = dict()
    for city in cities:
        city_fua = get_fua(fua, city)
        if city_fua is not None:
            bounds = city_fua.geometry.bounds.to_numpy()[0]
            bounds = [
                [bounds[3], bounds[0]],
                [bounds[1], bounds[2]]
            ]
            box = dict()
            fua_dic = dict()
            data[city]={}
            data[city]["box"] = bounds   
            data[city]["fua"] = list(city_fua.geometry.to_numpy()[0].geoms[0].exterior.coords)
            
    with open( filename, "w" ) as jsonFile:
        json.dump(data, jsonFile)

In [108]:
cities = [
    "Cairo",
    "Johannesburg",
    "Adana",
    "Ordu",
    "Trabzon",
    "Manisa"
]

In [109]:
write_json_fua("bounding_box.json", fua, cities)